In [6]:
import tweepy
from econfig import *
import datetime as dt

In [7]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

try:
    api.verify_credentials()
    print("Authentication Complete")
except:
    print("Authentication Unable to Complete")
    
two_days = (dt.datetime.now() - dt.timedelta(days=2)).strftime("%Y-%m-%d")
one_day = (dt.datetime.now() - dt.timedelta(days=1)).strftime("%Y-%m-%d")

# startDate = datetime.datetime(2014, 6, 1, 0, 0, 0)
# endDate =   datetime.datetime(2015, 1, 1, 0, 0, 0)
MAX_TWEETS = 5
tweets = tweepy.Cursor(api.search, q='#election2020', rpp=100, since='2020-02-20', until='2020-02-21').items(MAX_TWEETS)


Authentication Complete


In [8]:
id = []
username = []
followers_count = []
interaction_count = []
for tweet in tweets:
    interaction = tweet.retweet_count + tweet.favorite_count
    interaction_count.append(interaction)
    followers_count.append(tweet.user.followers_count)
    id.append(tweet.id)
    username.append(tweet.user.screen_name)
    pass


In [10]:
print(type(id[0]))

<class 'int'>


In [13]:
rows = zip(id,username,followers_count,interaction_count)

In [16]:
rows = zip(id,username,followers_count,interaction_count)
for row in rows:
    print(row)

(1230643261712322565, 'JohnVetsResist1', 6691, 0)
(1230643138756255744, 'Cant_Be_Passive', 55, 0)
(1230643029125607429, 'SheilaCreates', 1112, 0)
(1230642998670774275, 'AngeloJohnGage', 32155, 3)
(1230642936804773889, 'KVERNVOLLEN', 252, 0)


In [17]:
type(rows)

zip

In [18]:
type(row)


tuple